In [ ]:
import datetime as dt
import json
import re
import time

import datazimmer as dz
import pandas as pd

from aswan import get_soup, get_json, add_url_params, ConnectionError
from tqdm import tqdm

In [ ]:
fifa_rank_base = dz.SourceUrl("https://www.fifa.com/fifa-world-ranking/men")

In [ ]:
soup = get_soup(fifa_rank_base)

In [ ]:
dr = '"dates"\:\[.*?\]'

In [ ]:
date_ids = json.loads("{" + re.findall(dr, str(soup))[0] + "}")

In [ ]:
ranks_by_date = {}
for date_dic in tqdm(date_ids["dates"]):
    did = date_dic["id"]
    ranks_by_date[did] = get_json(
        "https://www.fifa.com/api/ranking-overview",
        params={"locale": "en", "dateId": date_dic["id"]},
    )
    time.sleep(0.1)

In [ ]:
uefa_seasons = list(range(2003, dt.date.today().year + 1))

In [ ]:
outer_keys = ["countryName", "displayName"]

all_ranks = []
for season_year in tqdm(uefa_seasons):
    for pref in [
        "coefficientType=MEN_CLUB&coefficientRange=OVERALL",
        "coefficientType=MEN_ASSOCIATION&coefficientRange=SEASONAL",
        "coefficientType=MEN_ASSOCIATION&coefficientRange=OVERALL",
    ]:
        url = (
            f"https://comp.uefa.com/v2/coefficients?{pref}&"
            f"seasonYear={season_year}&page=1&pagesize=500&language=en"
        )

        try:
            club_json = get_json(url)
        except ConnectionError:
            continue

        for member in club_json["data"]["members"]:
            outer_d = {k: member["member"][k] for k in outer_keys}
            for ranks in member["seasonRankings"]:
                all_ranks.append(outer_d | ranks)
    time.sleep(1)

In [ ]:
pd.DataFrame(all_ranks).to_csv(dz.get_raw_data_path("uefa-ranks.csv"), index=None)

In [ ]:
date_dic = {d["id"]: d["text"] for d in date_ids["dates"]}

In [ ]:
todrop = ["flag", "tag"]

In [ ]:
def flat(d0):
    d = d0.copy()
    ri = d.pop("rankingItem")
    return {k: v for k, v in (ri | d).items() if k not in todrop}

In [ ]:
fifa_dfs = []
for k, v in ranks_by_date.items():
    fifa_dfs.append(pd.DataFrame(map(flat, v["rankings"])).assign(date=date_dic[k]))

In [ ]:
pd.concat(fifa_dfs, ignore_index=True).to_csv(
    dz.get_raw_data_path("fifa-ranks.csv"), index=None
)